Chapter2
----------
텍스트 토큰화 하기
데이터셋 다운로드("The Verdict" - Edith Wharton )


In [1]:
import urllib.request

url = ("https://raw.githubusercontent.com/rickiepark/"
       "llm-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x21a873b13a0>)

Code 2-1 처음 99개 문자 출력

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("총 문자 개수: ", len(raw_text))
print(raw_text[:99])

총 문자 개수:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


텍스트를 토큰 리스트로 분할 ->re.split사용(문자열을 쪼개는 함수)



In [3]:
import re
text ="Hello, world. This, is a test"
#r'\s'가 아닌 r'(\s)'을 사용해서 공백도 출력 결과에 나옴
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test']


공백, 쉼표, 마침표도 분할하기

In [4]:
import re
text ="Hello, world. This, is a test"
#공백('\s')과 쉼표,마침표도 분할([,.])
result = re.split(r'([,.]/s)', text)
print(result)

['Hello, world. This, is a test']


중복된 공백 문자 처리하기

In [5]:
import re
text ="Hello, world. This, is a test"
'''
중복된 공백 문자 제거
item에서 빈 문자열은 False로 간주함
-> strip() 이후 내용이 남아 있으면 True: 남김
내용이 없으면 False: 버림
'''
result = [item for item in result if item.strip()]
print(result)

['Hello, world. This, is a test']


In [6]:
text = "Hello, world. Is this--a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [7]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocess = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

9235


In [8]:
print(preprocessed[:30])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--']


Section2.3
------------

토큰 ID를 만들려면 Vocabulary 구축이 필요함

이때 Vocabulary는 알파벳 순서로 정렬되어 있음

이전에 작업했던 preprocessed로 Vocabulary 사이즈를 확인


In [9]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1133


Vocabulary를 만들고 처음부터 51개 항목 출력하기

In [10]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i>= 50:
        break

('', 0)
('\n', 1)
(' ', 2)
('!', 3)
('"', 4)
("'", 5)
('(', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)
('At', 21)
('Be', 22)
('Begin', 23)
('Burlington', 24)
('But', 25)
('By', 26)
('Carlo', 27)
('Chicago', 28)
('Claude', 29)
('Come', 30)
('Croft', 31)
('Destroyed', 32)
('Devonshire', 33)
('Don', 34)
('Dubarry', 35)
('Emperors', 36)
('Florence', 37)
('For', 38)
('Gallery', 39)
('Gideon', 40)
('Gisburn', 41)
('Gisburns', 42)
('Grafton', 43)
('Greek', 44)
('Grindle', 45)
('Grindles', 46)
('HAD', 47)
('Had', 48)
('Hang', 49)
('Has', 50)


위의 출력 결과는 개별 토큰과 고유한 정수 레이블을 담고 있음


______

<아래코드>
LLM의 출력은 기본적으로 숫자로 되어있는데 이를 다시 텍스트로 바꾸는 작업이 필요함

->Inverse vocabulary가 필요함

In [11]:
class SimpleTokenizerV1:
    #self는 
    def __init__(self, vocab):
        self.str_to_int = vocab
        #inverse vocabular 만들기  
        self.int_to_str = {i:s for s,i in vocab.items()}

    #input text -> token ID
    def encode(self, text):
        #텍스트를 토큰 단위로 쪼갬 + strip으로 공백제거
        preprocessed = re.split(r'([,.;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        '''
        ids = []
        for s in preprocess:
        ids.append(self.str_to_int[s])
        ''' 
        #바로 위에 코드에서 vocab = {token: integer for integer, token in enumerate(all_words)}로 토큰ID를 이미 시켰음
        #enumerate로 이미 토큰화가 부여된 상태임
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    #Token ID -> Text
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        '''
        tokens = []
        for i in ids:               # ids 리스트 안의 숫자를 하나씩 꺼냄
        token = self.int_to_str[i]   # 숫자를 문자열 토큰으로 변환
        tokens.append(token)         # tokens 리스트에 넣음
        '''
        #문장 부호만 남기고 공백은 삭제하기
        #r'\1'는 
        text = re.sub(r'\s+([,.?"()\'])', r'\1', text)
        return text



위에서 만든것을 
새로운 text내용으로 Tokenizing 실행


In [12]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know,
         Mrs.Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[59, 5, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 70, 10, 41, 854, 1111, 757, 796, 10]


출력 결과는 아래 코드로 다시 복구 할 수 있음

In [13]:
print(tokenizer.decode(ids))

It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


만약 아래와 같은 문장을 수행하면?

In [14]:
#전체 실행을 위해 주석처리
#text = "Hello, do you like tea?"
#print(tokenizer.encode(text))

버그가 발생함 -> 위의 dataset에는 Hello라는 단어가 없음. ->Out of Vocabulary문제가 발생함

______

Section 2.4
------------
위에서 발생한 OOV문제를 해결하려면 Tokneizer를 수정해야함

예시로 토큰 2개를 추가: <|unk|>와 <|endoftext|>

<|unk|>: 모르는 단어를 만났을때 사용하도록 함

<|endoftext|>:  문서나 책 앞에 추가함(텍스트가 연결이 되어있으나, 관련이 없다는 것을 알려줌)

아래의 코드로 토큰을 추가

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))

1135


실제로 추가되었는지 확인하기 위해 Vocabulary의 마지막 5개 단어 출력

In [16]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1130)
('your', 1131)
('yourself', 1132)
('<|endoftext|>', 1133)
('<|unk|>', 1134)


위에서 추가한 토큰을 바탕으로 알지못하는 단어를 만났을떄 
<|unk|>토큰을 사용하도록 코드를 수정

In [17]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        #모르는 단어를 만났을때 <|unk|>토큰으로 바꾸기
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i]for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

텍스트가 끝났을때 <|endoftext|>를 사용하도록함

In [18]:
text1 = " Hello, do you like tea?"
text2 = " In the sunlit terraces of the palace."
text = "<|endoftext|>".join((text1, text2))
print(text)

 Hello, do you like tea?<|endoftext|> In the sunlit terraces of the palace.


위의 출력 결과를 보면 <|endoftext|>가 사용된 모습을 확인 할 수 있음

위에서 추가한 토큰을 적용해 다시 토큰화 실행

In [19]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1134, 8, 358, 1129, 631, 978, 13, 1133, 58, 991, 959, 987, 725, 991, 1134, 10]


아래의 코드로 실제 토큰이 잘 추가 되었는지 확인

In [20]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


위에서는 버그가 나왔던 Hello가 지금은 <|unk|>로 대체되어 버그 없이 잘 실행됨

여기에서 사용된 특수 토큰 외에 다른 토큰들에는

[BOS]: Beginning of sequence - 텍스트의 시작 부분 표시

[EOS]: End of sequence -  텍스트의 끝에 위치 e.g. 서로다른 문서와 책을 합칠 경우 사용

[PAD]: Padding - 토큰을 사용해 batch에서 가장 긴 텍스트 길이까지 확장할때 사용

그런데 GPT에서는 <|endoftext|>만 사용하며 <|unk|>도 사용 안함
_______

Section2.5
---------
BPE(Byte Pair Encoding)을 사용하기 위해 tiktoken 사용

In [21]:

pip install tiktoken


Note: you may need to restart the kernel to use updated packages.


tiktoken을 사용해서 이전에 사용한 구문 encode해보기

In [22]:
from importlib.metadata import version
import tiktoken
#BPE토크나이저 초기화
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace"
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271]


다시 decode를 사용해서 복구도 할 수 있음

In [23]:
strings= tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace


만약 모르는 단어가 나온다면?

In [24]:
text = "Sunhong Min"
praticeEncoder = tokenizer.encode(text)
print(praticeEncoder)
praticeDecoder = tokenizer.decode(praticeEncoder)
print(praticeDecoder)


[16012, 71, 506, 1855]
Sunhong Min


위는 encode 아래는 decode 출력결과인데 사전에 없는 단어로도 잘 나오는 모습을 볼 수 있음

BPE Tokenizer는 <|unk|> 토큰을 안쓰는데도 someunknownPlace처럼 모르는 단어를 인코딩, 디코딩을 잘함 

모르는 단어를 개별 문자나 부분 단어로 토큰화해서 모든 텍스트를 처리 할 수 있음

______
책에서 설명하는 BPE는 개별 문자 e.g "a", "b"를 Vocabulary에 추가하고 자주 등장하는 단어끼리 합침

e.g. define, depend .. 처럼 de가 자주 등장하면 de로 합침 .. 

합치는 기준은 determined by a frequency cutoff 라고함

추가설명은 ppt
_____
Section2.6
--------

LLM은 다음 단어를 예측하는 식으로 훈련됨 ->Sliding window사용

​

BPE토크나이저로 "the Verdict"전체 토큰화 시키기

In [25]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


실행하면 5145가 나오는데  훈련세트에 있는 총 토큰 개수 출력한 것임

more interesting text passage in the next steps하기 위해서 처음 50개 토큰을 삭제한다고 함

In [26]:
enc_sample = enc_text[50:]

다음 단어 예측(input- target)을 만들기 위해서 x, y변수를 사용함

x: 입력 토큰 담기

y: 토큰 하나만큼 이동한 타깃 담기



단순히 y가 x보다 한 칸 더 window를 이동시킨 것을 출력하는 코드

In [27]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y        {y}")

x: [290, 4920, 2241, 287]
y        [4920, 2241, 287, 257]


___
화살표 왼쪽: LLM이 입력 받은 값

화살표 오른쪽: LLM이 예측해야 할 토큰 ID


In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


토큰ID를 Text로 바꾸기

In [29]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [30]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


토큰 -> 임베딩 변환을 위해 데이터 로더 구현 필요

데이터로더: input data를 순회하며 input과 target을  pytorch tensor로 tensor반환

e.g. tensor x,y 가 있다고 하면

tensor x: 입력을 모아둠

tensor y: predict target을 모아둠

dataset class를 만들기

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatabaseV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        #전체 텍스트 토큰화
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            #input 시퀀스
            input_chunk = token_ids[i:i + max_length]
            #정답 시퀀스 - input보다 한 칸 오른쪽으로 이동
            target_chunk = token_ids[i + 1: i + max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    

    #데이터셋에 있는 전체 행 수를 반환
    def __len__(self):
        return len(self.input_ids)
    
    #데이터셋에서 하나의 행을 반환
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


input -target pair의 batch를 생성하기

In [32]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride = 128, 
                         shuffle=True, 
                         #drop_last를 True로 둠
                         #전체 Data size를 batch_size로 나누었을때 마지막에 안나누어 떨어지는 batch를 버림
                         drop_last = True,
                         num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatabaseV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader



이후 max_lenght=4로 batch_size = 1로 잘 작동하는지 확인하기 

In [33]:
with open("the-verdict.txt", "r", encoding='utf-8')as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


첫번째 tensor는 input token ID를 저장, 두 번째 텐서는 target token ID를 저장

현재 max_length는 4인데, 보통 256을 사용함

​

pratice problem)max_length와 stride의 크기 변경해보기

In [34]:
def create_dataloader_pratice(txt, batch_size=4, max_length=256,
                         stride = 2, 
                         shuffle=True, 
                         drop_last = True,
                         num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatabaseV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader_pratice

with open("the-verdict.txt", "r", encoding='utf-8')as f:
    raw_text = f.read()

dataloader_pratice = create_dataloader_v1(
    raw_text, batch_size=19, max_length=3, stride=1, shuffle=False
)

data_iter = iter(dataloader_pratice)
first_batch = next(data_iter)
print(first_batch)



[tensor([[   40,   367,  2885],
        [  367,  2885,  1464],
        [ 2885,  1464,  1807],
        [ 1464,  1807,  3619],
        [ 1807,  3619,   402],
        [ 3619,   402,   271],
        [  402,   271, 10899],
        [  271, 10899,  2138],
        [10899,  2138,   257],
        [ 2138,   257,  7026],
        [  257,  7026, 15632],
        [ 7026, 15632,   438],
        [15632,   438,  2016],
        [  438,  2016,   257],
        [ 2016,   257,   922],
        [  257,   922,  5891],
        [  922,  5891,  1576],
        [ 5891,  1576,   438],
        [ 1576,   438,   568]]), tensor([[  367,  2885,  1464],
        [ 2885,  1464,  1807],
        [ 1464,  1807,  3619],
        [ 1807,  3619,   402],
        [ 3619,   402,   271],
        [  402,   271, 10899],
        [  271, 10899,  2138],
        [10899,  2138,   257],
        [ 2138,   257,  7026],
        [  257,  7026, 15632],
        [ 7026, 15632,   438],
        [15632,   438,  2016],
        [  438,  2016,   257],
     

____
batch_size>1일 경우 샘플링하는 코드

In [35]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Input: ", inputs)
print("\n Target:", targets)

Input:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Target: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


Cf) batch_size에 중첩이 있으면 과적합이 생길 가능성이 있음

윈도우 1: [1,2,3,4]
윈도우 2:   [2,3,4,5]
윈도우 3:     [3,4,5,6]
각 시퀀스가 75%이상 겹침 -> 중복되는 문장을 반복적으로 학습하게됨 ->Overfitting
_____
Section2.7
----------

Input token이 4개 있다고 가정 후 임베딩층 가중치 행렬 추가하는 코드

In [36]:
input_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


가중치 행렬은 랜덤한 값을 갖고 있는데, 이는 LLM훈련할때 최적화됨

위에 코드에서 보면 
vocab_size = 6
output_dim = 3

으로 설정했는데 이때문에 행이6개 열이3개가 나옴

위의 코드를 토큰 ID에 적용하면


In [37]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


tensor의 4번째 행과 동일한 결과가 나옴.

->embedding layer는 토큰ID를 기반으로 weight tensor에서 행을 추출함

​

위에서 입력한 4개의 input id를 모두 적용하면 아래의 결과가 나옴

In [38]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


사전 정의된 Vocabulary(dataset)에서 필요한 행만 뽑아서 토큰 임베딩 결과를 만든다

​

cf) 데이터셋 = 해리포터안에 있는 단어를 토큰화 - > 임베딩을 시킴

근데 새로운 단어 fox jmps over dog라는 문장 있다고 가정

-> 해리포터에서 저 단어4개가 이미 토큰화가 되었을거임

-> fox jumps over dog에 알맞는 토큰값을 해리포터 Vocabulary dictionary에서 찾음

-> fox jumps over dog에맞는 토큰id를 얻게됨

____

Section2.8
-------


LLM은 위치에 대한 개념이 없음 정확하게는 self-attention에게 없음

​

cf)Attention is all you need 논문에서는 삼각함수로 positional encoding을 진행함
PPT설명: positional? +  positional encoding이 필요한 이유?

​

Positional Encoding은 상대적인 위치임 ->멀리 떨어져 있는 정도로 관계를 학습

->일반화에 좋음

​

50257크기의 Vocabulary dictionary로 BPE토크나이저 진행

In [39]:
import torch

vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

​

샘플링한 각 배치에 있는 토큰 ID를 256차원 벡터로 임베딩

if) batchsize:8, token:4일경우 8 * 4 * 256 tensor가 나옴

In [40]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token ID: ", inputs)
print("\n Inputsize:\n", inputs.shape)

Token ID:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Inputsize:
 torch.Size([8, 4])


여기서 torch.Size가([8, 4])인데 batch에 4개의 토큰을 가진 text sample 8개가 들어있다는 의미임

batch 1 step을 보여줌
_______

임베딩 층을 사용해 토큰 ID를 256차원 벡터로 임베딩

In [41]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


위의 출력 결과는

([batch_size, sequence length(max_length), embedding dimension

을 나타냄

-> 이는 각 토큰ID가 256차원 벡터로 임베딩 된 것을 보여줌

___________
GPT의 절대 임베딩 

-> token_embedding_layer와 동일한 임베딩 차원을 가지는 또다른 임베딩 층을 만들면 됨

GPT계열 모델은 2종류의 임베딩을 더해서 최종 입력 벡터를 만들음

token embedding과 absolute positional embedding

두 임베딩을 더해야하기때문에 두 차원이 동일해야함

In [42]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


위의 출력 결과는

[(sequence length, embedding dimension)]

을 의미함

_______

둘이 더하면 positional embedding 끝

In [43]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


Chapter3
-----------

In [44]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], #Your (x^1)
    [0.55, 0.87, 0.66], #journey (x^2)
    [0.57, 0.85, 0.64], #starts (x^3)
    [0.22, 0.58, 0.33], #with (x^4)
    [0.77, 0.25, 0.10], #one (x^]5)
    [0.05, 0.80, 0.55]] #step (x^6)
)

In [45]:
query = inputs[1]
attn_score_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(x_i, query)
print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [46]:
attn_weight_2_tmp = attn_score_2 / attn_score_2.sum()
print("Attention Weight: ", attn_weight_2_tmp)
print("Sum: ", attn_weight_2_tmp.sum())

Attention Weight:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [47]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weight_2_naive = softmax_naive(attn_score_2)
print("attention weight: ", attn_weight_2_naive)
print("Sum: ", attn_weight_2_naive.sum())


attention weight:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [48]:
attn_weight_2 = torch.softmax(attn_score_2, dim = 0)
print("attention weight: ", attn_weight_2)
print("sum: ", attn_weight_2.sum())

attention weight:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum:  tensor(1.)


In [49]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weight_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [50]:
attn_score = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_score[i,j] = torch.dot(x_i, x_j)
print(attn_score)
        

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [51]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [52]:
attn_weights = torch.softmax(attn_scores, dim = -1)
print(attn_scores.shape)
print(attn_weights)

torch.Size([6, 6])
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [53]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("2번째 행의 합: ", row_2_sum)
print("모든 행의 합: ", attn_weights.sum(dim=-1))

2번째 행의 합:  1.0
모든 행의 합:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [54]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [55]:
print("3.3.1 row2 context vector: ", context_vec_2)

3.3.1 row2 context vector:  tensor([0.4419, 0.6515, 0.5683])


In [56]:
#2번째 입력 원소
x_2 = inputs[1]
#input embedding size, d_in = 3
d_in = inputs.shape[1]
#output embedding size, d_out = 2
d_out = 2

In [57]:
torch.manual_seed(123)
#requires_grad: backpropagation의 수행 여부
#실제 훈련시에는 True로 지정 -> weight mat 업데이트
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key	= torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value	= torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [58]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [59]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape: ", keys.shape)
print("values.shape: ", values.shape)

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


In [60]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [61]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [62]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim =-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [63]:
context_vec_2 = attn_weight_2 @ values 
print(context_vec_2)

tensor([0.3069, 0.8188])


In [64]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
	def __init__(self, d_in, d_out):
		super().__init__()
		self.W_query = nn.Parameter(torch.rand(d_in, d_out))
		self.W_key 	 = nn.Parameter(torch.rand(d_in, d_out))
		self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        
	def forward(self,x):
		keys= x @ self.W_key
		queries = x @ self.W_query
		values = x @ self.W_value
		attn_scores = queries @ keys.T #omega
		attn_weights = torch.softmax(
			attn_scores / keys.shape[-1]**0.5, dim=-1
		)
		context_vec = attn_weights @ values
		return context_vec


In [65]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [66]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super(). __init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key 	 = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self,x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec
           
        
        

In [67]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))


tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [68]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [69]:
context_length = attn_scores.shape[0]
#trill은 lower triangle = 대각선보다 위쪽을 0으로 만들음
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [70]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [71]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [ ]:
#diagonal=1 주대각선보다 한 칸 위를 -inf로 설정
mask = torch.triu(torch.ones(context_length, context_length),diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [77]:
#마스킹된 결과에 softmax 적용
#shape[-1] : 텐서의 마지막 축 크기
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [76]:
#컨텍스트 벡터 출력하기
context_vec = attn_weights @ values
print(context_vec)

tensor([[0.1855, 0.8812],
        [0.2795, 0.9361],
        [0.3133, 0.9508],
        [0.2994, 0.8595],
        [0.2702, 0.7554],
        [0.2772, 0.7618]], grad_fn=<MmBackward0>)


In [78]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [79]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)
